지금까지는 **Sequential** 모델을 사용하여 만들었습니다.
* 네트워크 입력과 출력이 하나라고 가정합니다
* 이 모델은 층을 차례대로 쌓아 구성합니다

<br></br>
![](../images/7-1-sequential.png)
<br></br>

일부 네트워크는 *개별 입력이 여러개* 필요하거나 *출력이 여러개* 필요합니다. 층을 여러개로 쌓지 않고 층 사이를 연결하여 그래프처럼 만드는 네트워크도 있습니다.